## 개요

> 내가 좋아하는 리듬 게임이 있는데, 
> 이 게임은 점수를 뽑아올 수 있는 사이트가 있다.
> 이 사이트에서 곡을 하나씩 찾아 입력해서 점수를 확인 하는 방법은 좀 번거롭다
> 그래서 이렇게 메크로를 만들어 보았다.

## 필요한 모듈 호출

In [1]:
from bs4 import BeautifulSoup as bs

## 게임에서 자신의 아이디 입력

In [2]:
user_id=input("Input your ID >")

Input your ID >776114322


## 곡 목록 가져오기

In [3]:
import requests as req

In [4]:
import json

In [5]:
from tqdm import tqdm_notebook

In [6]:
res = req.get("https://lab.zl-studio.cn/potential/ArcTools/ArcSonglist.json")

In [7]:
song_dic = json.loads(res.text)['songs']

In [8]:
song_info = []
for song in song_dic:
    song_tmp=[]
    song_tmp.append(song['id'])
    song_tmp.append(song['title_localized']['en'])
    for i in range(3):
        song_tmp.append([song['difficulties'][i]['rating'],song['difficulties'][i]['fixedValue']])
    song_info.append(song_tmp)
    

## 크롤링 실행

In [10]:
song_rlt_list = []
difficulty = ['PAST', 'PRESENT', 'FUTURE']
for i in tqdm_notebook(range(len(song_info) * 3)):
    res = req.get("https://lab.zl-studio.cn/potential/search.php?Aid=%s&songid=%s&difficulty=%d" % (user_id, song_info[i // 3][0], i % 3))
    song_dict={}
    song_dict['song_name'] = song_info[i // 3][1]
    song_dict['difficulty'] = difficulty[i % 3]
    song_dict['level'] = song_info[i // 3][i % 3 + 2][0]
    song_dict['detail_level'] = song_info[i // 3][i % 3 + 2][1]
    if "You hadn't played this song!" in res.text:
        song_dict['played']=False
        song_rlt_list.append(song_dict)
    else:
        res_dict=json.loads(res.text)
        song_dict['played']=True
        for i in ['score', 'shiny_perfect_count', 'perfect_count', 'near_count', 'miss_count', 'best_clear_type']:
            song_dict[i]=res_dict[i] 
        song_rlt_list.append(song_dict)
print(song_rlt_list)

[{'song_name': 'Sayonara Hatsukoi', 'difficulty': 'PAST', 'level': 1, 'detail_level': 1.5, 'played': True, 'score': 10000202, 'shiny_perfect_count': 202, 'perfect_count': 205, 'near_count': 0, 'miss_count': 0, 'best_clear_type': 3}, {'song_name': 'Sayonara Hatsukoi', 'difficulty': 'PRESENT', 'level': 4, 'detail_level': 4.5, 'played': True, 'score': 10000301, 'shiny_perfect_count': 301, 'perfect_count': 305, 'near_count': 0, 'miss_count': 0, 'best_clear_type': 3}, {'song_name': 'Sayonara Hatsukoi', 'difficulty': 'FUTURE', 'level': 6, 'detail_level': 6.5, 'played': True, 'score': 10000629, 'shiny_perfect_count': 629, 'perfect_count': 666, 'near_count': 0, 'miss_count': 0, 'best_clear_type': 3}, {'song_name': 'Lost Civilization', 'difficulty': 'PAST', 'level': 4, 'detail_level': 4, 'played': True, 'score': 10000453, 'shiny_perfect_count': 453, 'perfect_count': 462, 'near_count': 0, 'miss_count': 0, 'best_clear_type': 3}, {'song_name': 'Lost Civilization', 'difficulty': 'PRESENT', 'level':

In [11]:
import csv

In [12]:
with open('arcaea result.csv', 'w', newline="\n", encoding='utf-8') as csv_f:
    fieldnames = ['song_name', 'difficulty', 'level', 'detail_level', 'played', 'score', 'shiny_perfect_count', 'perfect_count', 'near_count', 'miss_count', 'best_clear_type']
    writer = csv.DictWriter(csv_f, fieldnames=fieldnames)
    writer.writeheader()
    for song in song_rlt_list:
        writer.writerow(song)